In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics

from tqdm import tqdm
from torchvision.transforms import v2
from torch.utils.data import DataLoader, Dataset
from torchvision.models import efficientnet_b7, EfficientNet_B7_Weights, resnet50, EfficientNet_B7_Weights
from PIL import Image

sns.set(style="whitegrid", palette="Set2")

In [2]:
# Try SMOTE? (Not ideal, it sucks)
# Try Cost-Sensitive Learning 
# Get from Original Dataset

In [3]:
PATH = "/kaggle/input/tammathon-task-2"

train_df = pd.read_csv(f"{PATH}/train.csv")
valid_df = pd.read_csv(f"{PATH}/val.csv")
test_df = pd.read_csv(f"{PATH}/test.csv")

In [4]:
train_df.iloc[0]['path'].split('/')[0]

'train'

In [49]:
class ICAODataset(Dataset):
    def __init__(self, root_dir, dataframe, transforms=None):
        self.root_dir = root_dir
        self.dataframe = dataframe.reset_index(drop=True)
        self.transforms = transforms
        self.subset = dataframe.iloc[0]['path'].split('/')[0]

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        img_path = os.path.join(self.root_dir, self.subset, row['path'])
        img = Image.open(img_path).convert("RGB")  # ensure 3 channels
        
        label = torch.tensor(row['label'], dtype=torch.float32)  # binary case
        if self.transforms:
            img = self.transforms(img)
        return img, torch.tensor([label])


train_transforms = v2.Compose([
    v2.Resize(600),
    v2.RandomHorizontalFlip(p=0.3),
    v2.ColorJitter(0.4, 0.4, 0.4, 0.4),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], 
                 std=[0.229, 0.224, 0.225])
])

valid_transforms = v2.Compose([
    v2.Resize(600),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], 
                 std=[0.229, 0.224, 0.225])
])

train_data = ICAODataset(PATH, train_df, transforms=train_transforms)
valid_data = ICAODataset(PATH, valid_df, transforms=valid_transforms)

train_loader = DataLoader(train_data, num_workers=0, shuffle=True, 
                          pin_memory=True)
valid_loader = DataLoader(valid_data, num_workers=0, shuffle=False, 
                          pin_memory=True)

In [50]:
class EffNetB7Base(nn.Module):
    def __init__(self, n_classes=1):
        super().__init__()
        model = efficientnet_b7(weights=EfficientNet_B7_Weights.IMAGENET1K_V1)
        in_features = model.classifier[1].in_features
        
        self.backbone = nn.Sequential(*list(model.children())[:-1])
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(in_features, 1)

    def forward(self, x):
        out = self.backbone(x)
        out = self.pool(out)
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out  # raw logits


class ResNet50Base(nn.Module):
    def __init__(self, n_classes=1):
        super().__init__()
        model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
        in_features = model.fc.in_features

        # Remove the original classifier
        self.backbone = nn.Sequential(*list(model.children())[:-1])
        self.classifier = nn.Linear(in_features, n_classes)

    def forward(self, x):
        out = self.backbone(x)
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out

In [51]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        probs = torch.sigmoid(inputs)
        ce_loss = F.binary_cross_entropy(probs, targets, reduction='none')
        pt = torch.exp(-ce_loss)  # pt = e^(-BCE)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        
        return focal_loss.mean()

In [52]:
imgs, labels = next(iter(train_loader))

In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = {
    "lr": 1e-4,
    "weight_decay": 1e-4
}

effnet_model = EffNetB7Base(1).to(device)
class_weights = torch.tensor([0.67, 0.33]).to(device)

focal_loss = FocalLoss(alpha=0.75)
bce_loss = nn.BCEWithLogitsLoss(pos_weight=class_weights)

optimizer = torch.optim.AdamW(effnet_model.parameters(), 
                              lr=config['lr'],
                              weight_decay=config['weight_decay'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", patience=2) # So LR doesn't plateau smh
metric_fn = torchmetrics.F1Score(task='binary')

In [ ]:
# For Reference: a CrossValidator class I created back then

class CrossValidator:
    def __init__(self, 
                 models, 
                 metric_fns, 
                 cv_method, 
                 scaler = None, 
                 name = None, 
                 pi_kwargs = None, 
                 pred_probs = False, 
                 verbose = True):
        """
           A class for performing cross-validation on a set of models with various metric functions.

           Attributes:
               models (list): A list of tuples containing the model name and the model object.
               metric_fns (list): A list of tuples containing the metric name and the metric function.
               cv_method (object): A cross-validation method object from scikit-learn.
               scaler (object, optional): A scaler object from scikit-learn to scale the data.
               name (str, optional): A name for the cross-validator.
               pi_kwargs (dict, optional): A dictionary of keyword arguments for permutation importance calculation.
               pred_probs (bool, optional): Whether to predict probabilities or class labels.
               perm_imp (dict, optional): A dictionary containing permutation importances for each model.
               oof_preds (dict, optional): A dictionary containing out-of-fold predictions for each model.
               oof_metrics (dict, optional): A dictionary containing out-of-fold metric scores for each model.
               data (tuple, optional): A tuple containing the test features and labels used in each fold.
               oof_metrics_df (pd.DataFrame, optional): A pandas DataFrame containing the mean out-of-fold metric scores for each model.

           Methods:
               _calculate_metrics(y_test, y_pred): Calculates and prints the metric scores for a given set of true labels and predicted labels.
               _cross_validate(X, y): Performs cross-validation on the given data and models, and stores the out-of-fold predictions, metric scores, and permutation importances.
               _get_oof_metrics(metric_dict): Converts the out-of-fold metric scores dictionary to a pandas DataFrame.
               fit(X, y): Initiates the cross-validation process and stores the results.
       """
        
        self.name = name
        self.verbose = verbose
        
        self.models = models
        self.metric_fns = metric_fns
        self.cv_method = cv_method
        self.scaler = scaler
        self.pi_kwargs = pi_kwargs
        self.pred_probs = pred_probs
        
        self.perm_imp = None
        self.oof_preds = None
        self.oof_metrics = None
        self.data = None
        
        self.oof_metrics_df = None
    
    def _calculate_metrics(self, y_test, y_pred):
        # Dictionary to store the score for each metric
        results = {}

        # Loop through each metric
        for metric_fn in self.metric_fns:
            # Calculate score using metric
            if metric_fn[0] == 'ROC AUC':
                score = metric_fn[1](y_test, y_pred, multi_class = 'ovr')
            else:
                score = metric_fn[1](y_test, y_pred)

            # Store score as value and metric as key
            results[metric_fn[0]] = score
            
            if self.verbose:
                # Display metric score
                print(f'{metric_fn[0]} : {score:.5f}\n')
        
        return results

    def _cross_validate(self, X, y):
        # Dictionaries to store out-of-fold predictions and out-of-fold metric scores
        oof_preds, oof_metrics = {}, {}
        
        # Lists to aggregate test features and labels used in each fold
        x_data, y_data = [], []
        
        # Dictionary to store permutation feature importance for each fold
        perm_imp = {}
        
        if self.verbose:
            print(f'Name: {self.name} | {self.cv_method.n_splits}-Fold\n')
        
        for idx, (train_idx, test_idx) in enumerate(self.cv_method.split(X, y)):
            if self.verbose:
                print(f'Fold {idx}:')
                print('-'*40+'\n')
            
            x_train, x_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            x_data.extend(x_test)
            y_data.extend(y_test)
            
            if self.scaler is not None:
                x_train = self.scaler.fit_transform(x_train)
                x_test = self.scaler.transform(x_test)
            
            for model in self.models:
                if self.verbose:
                    print(f'Cross-validating: [{model[0]}]\n')
                
                model[1].fit(x_train, y_train)
                
                if model[0] not in oof_preds:
                    oof_preds[model[0]] = []
                    oof_metrics[model[0]] = {}                    
                    perm_imp[model[0]] = []
                
                y_pred = model[1].predict_proba(x_test) if self.pred_probs else model[1].predict(x_test)
                
                # Save model predictions
                oof_preds[model[0]].append(y_pred)
                
                # Calculate metrics
                for metric_name, result in self._calculate_metrics(y_test, y_pred).items():
                    # Save metric result if not already in dictionary
                    if metric_name not in oof_metrics[model[0]]:
                        oof_metrics[model[0]][metric_name] = []
                    oof_metrics[model[0]][metric_name].append(result)
                
                # Calculate permutation importances
                if self.pi_kwargs is not None:
                    if self.verbose:
                        print(' -- Calculating Permutation Importances...\n')
                    perm_result = permutation_importance(model[1], x_test, y_test, **self.pi_kwargs)
                    perm_imp[model[0]].append(perm_result.importances)
        
        if self.pi_kwargs is not None:
            self.perm_imp = perm_imp
        
        return oof_preds, oof_metrics, (x_data, y_data)
    
    def _get_oof_metrics(self, metric_dict):
        dict, df = {'models' : []}, pd.DataFrame()
        
        for model in list(metric_dict.keys()):
            dict['models'].append(model)
            for metric in list(metric_dict[model].keys()):
                if metric not in dict:
                    dict[metric] = []
                dict[metric].append(np.mean(metric_dict[model][metric]))

        df.index = dict['models']
        for metric in list(dict.keys())[1:]:
            df[metric] = dict[metric]

        return df      
    
    def fit(self, X, y):
        self.oof_preds, self.oof_metrics, self.data = self._cross_validate(X, y)
        self.oof_metrics_df = self._get_oof_metrics(self.oof_metrics)

In [55]:
def train_model(
    model,
    train_loader, 
    loss_fn,
    optimizer,
    metric_fn,
    device,
    scheduler=None,
):
    model.train()
    running_metric = 0.0
    running_loss = 0.0

    with tqdm(train_loader, unit="batch", desc="Training") as progress_bar:
        for batch, (x, y) in enumerate(progress_bar):
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            
            yhat = model(x) # Model returns logits,
            yhat = torch.sigmoid(yhat) # Apply sigmoid for pred probs !!! 

            loss = loss_fn(yhat, y)
            loss.backward()
            optimizer.step()

            metric_fn.update(yhat, y)
            running_loss += loss.item()
            running_metric += metric_fn.compute().item()

            progress_bar.set_postfix(loss=loss.item(), 
                                     metric=running_metric / (batch + 1))

    epoch_loss = running_loss / len(train_loader)
    epoch_metric = running_metric / len(train_loader)

    metric_fn.reset()

    print(f"\nTraining Loss: {epoch_loss:.4f}, F1 Score: {epoch_metric:.4f}")
    return epoch_loss, epoch_metric


In [56]:
def validate_model(
    model,
    valid_loader,
    loss_fn,
    metric_fn,
    device
):
    model.eval()
    running_loss = 0.0
    running_metric = 0.0

    with torch.inference_mode():
        with tqdm(valid_loader, unit="batch", desc="Validation") as progress_bar:
            for x, y in progress_bar:
                x, y = x.to(device), y.to(device)

                yhat = model(x)
                yhat = torch.sigmoid(yhat)

                loss = loss_fn(yhat, y)

                metric_fn.update(yhat, y)

                running_loss += loss.item()
                running_metric += metric_fn.compute().item()

                progress_bar.set_postfix(loss=loss.item(), 
                                         metric=running_metric / (len(progress_bar) + 1))

    epoch_loss = running_loss / len(valid_loader)
    epoch_metric = running_metric / len(valid_loader)

    metric_fn.reset()

    print(f"\nValidation Loss: {epoch_loss:.4f}, F1 Score: {epoch_metric:.4f}")
    return epoch_loss, epoch_metric


In [57]:
def training_pipeline(
    epochs,
    model,
    train_loader,
    valid_loader,
    loss_fn,
    metric_fn,
    optimizer,
    device,
    scheduler=None,
):
    train_losses, valid_losses = [], []
    train_metrics, valid_metrics = [], []
    
    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs}")

        train_loss, train_metric = train_model(
            model, train_loader, loss_fn, optimizer, metric_fn, device, scheduler
        )
        valid_loss, valid_metric = validate_model(
            model, valid_loader, loss_fn, metric_fn, device
        )

        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        train_metrics.append(train_metric)
        valid_metrics.append(valid_metric)

        if scheduler:
            scheduler.step(valid_loss)

        print(f"Train Loss: {train_loss:.4f}, Train Metric: {train_metric:.4f}")
        print(f"Validation Loss: {valid_loss:.4f}, Validation Metric: {valid_metric:.4f}")

    return train_losses, valid_losses, train_metrics, valid_metrics

In [58]:
train_losses, valid_losses, train_metrics, valid_metrics = training_pipeline(
    10, effnet_model, train_loader, valid_loader, focal_loss, metric_fn, optimizer, device, scheduler
)


Epoch 1/10


Training:   0%|          | 5/3978 [00:41<9:09:44,  8.30s/batch, loss=0.269, metric=0] 


KeyboardInterrupt: 